In [ ]:
pip install langchain sentence-transformers faiss-cpu


In [ ]:
pip install -U langchain-community

In [ ]:
# rag_pipeline_fastapi.py

# 1. Corpus Preparation
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.document_loaders import TextLoader

import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/India"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Get all paragraphs
paragraphs = [p.get_text() for p in soup.find_all('p')]
corpus_text = " ".join(paragraphs)

# Now split corpus_text into chunks (as before)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunked_texts = splitter.split_text(corpus_text)

In [ ]:
# 2. Embedding Generation & Vector Store
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Instantiate HuggingFace sentence-transformer for text embeddings
embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# Create a FAISS vector store
vectorstore = FAISS.from_texts(chunked_texts, embedder)

/tmp/ipython-input-3352903718.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
pip install langchain google-generativeai langchain-google-genai

In [ ]:
# # 3. Retriever + Large Language Model (LLM) Pipeline
from langchain.chains import RetrievalQA
# from langchain.llms import HuggingFacePipeline
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# # Load HuggingFace LLM for response generation (small, fast model here)
# model_id = "distilgpt2"
# llm_model = AutoModelForCausalLM.from_pretrained(model_id)
# llm_tokenizer = AutoTokenizer.from_pretrained(model_id)
# llm_model.config.max_length = 1024  # or larger depending on your longest input

# # Increased max_length to avoid ValueError
# generator = pipeline(
#     "text-generation",
#     model=llm_model,
#     tokenizer=llm_tokenizer,
#     max_new_tokens=150,
#     pad_token_id=llm_tokenizer.eos_token_id  # Prevent warnings with GPT-2
# )


# # Wrap in LangChain LLM interface
# llm = HuggingFacePipeline(pipeline=generator)

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from langchain.llms import HuggingFacePipeline

model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

generator = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150
)

llm = HuggingFacePipeline(pipeline=generator)

# Build RetrievalQA pipeline
qa_pipeline = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True  # optional, for traceability
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# 7. Query the pipeline
def ask_question(question: str):
    # Use 'invoke' to get the result when there are multiple output keys
    result = qa_pipeline.invoke(question)

    print("\nQuestion:", question)
    print("Answer:", result['result'] if isinstance(result, dict) and 'result' in result else result)
    return result['result'] if isinstance(result, dict) and 'result' in result else result

# Example usage
if __name__ == "__main__":
    sample_questions = [
        "What is the capital of India?",
    ]
    for q in sample_questions:
        ask_question(q)


Question: What is the capital of India?
Answer: Delhi


Evaluation

In [ ]:
pip install rouge_score

In [ ]:
import evaluate

# Initialize ROUGE metric
rouge = evaluate.load('rouge')

# Ground truth answers aligned with queries (as above)
references = [
    "Republic of India",
    "South Asia",
    "Seventh-largest country by area",
    "1947",
    "Indian Ocean, Arabian Sea, Bay of Bengal",
    "Pakistan, China, Nepal, Bhutan, Bangladesh, Myanmar",
    "Over 1.4 billion in 2023",
    "Hindi, English, and 21 other scheduled languages",
    "Federal parliamentary democratic republic",
    "Parliamentary system",
    "Narendra Modi",
    "Indus Valley Civilization, Maurya and Gupta empires, Buddhism and Jainism emergence",
    "Hinduism, Islam, Christianity, Sikhism, Buddhism, Jainism",
    "One of the earliest urban cultures in South Asia",
    "Maurya Empire, Gupta Empire, Mughal Empire",
    "British colonial rule ended in 1947; economic and social changes",
    "28 states and 8 union territories",
    "Around 74% literacy rate as of 2011",
    "Service sector, industry, and agriculture",
    "Fifth-largest nominal GDP (2024), third-largest by PPP",
    "Poverty, economic inequality, gender inequality, child malnutrition, air pollution",
    "Influenced by Himalayas and Thar Desert; monsoons",
    "One of the world’s megadiverse countries with four biodiversity hotspots",
    "Third-largest pharmaceutical producer and largest supplier of generic medicines",
    "Bicameral Parliament with Rajya Sabha and Lok Sabha"
]

def evaluate_rouge(candidates, references=references):
    """
    candidates: List of answer strings returned by your RAG system aligned with references order.
    references: List of ground truth answers.

    Prints ROUGE scores (ROUGE-1, ROUGE-2, ROUGE-L) averaged over all queries.
    """
    results = rouge.compute(predictions=candidates, references=references)
    print("ROUGE evaluation results:")
    print(f"ROUGE-1 (F1): {results['rouge1']:.4f}")
    print(f"ROUGE-2 (F1): {results['rouge2']:.4f}")
    print(f"ROUGE-L (F1): {results['rougeL']:.4f}")

# Example usage:
# Suppose rag_answers is a list of your RAG system's answers for the 25 queries
# rag_answers = [ "Republic of India", "South Asia", ..., "Bicameral Parliament with Rajya Sabha and Lok Sabha" ]
# evaluate_rouge(rag_answers)


In [ ]:
question = [
    "What is the official name of India?",
    "Where is India located geographically?",
    "How large is India compared to other countries?",
    "When did India gain independence?",
    "What seas and oceans surround India?",
    "Which countries share land borders with India?",
    "What is the population of India as of 2023?",
    "What are the major languages spoken in India?",
    "How is India politically structured?",
    "What type of government does India have?",
    "Who is the current Prime Minister of India?",
    "What were the key events during ancient India?",
    "What are the major religions practiced in India?",
    "What is the significance of the Indus Valley Civilization?",
    "Which empires ruled India historically?",
    "What were the effects of British colonial rule in India?",
    "What are the major administrative divisions in India?",
    "What is the literacy rate in India?",
    "What are the key economic sectors of India?",
    "How has India’s GDP changed over the years?",
    "What are the main challenges India faces socially?",
    "How is India’s climate influenced?",
    "What is the biodiversity significance of India?",
    "How does India contribute to the global pharmaceutical industry?",
    "What is the role of the Indian Parliament?"
]

In [ ]:
candidates = []
for q in question:
        candidates.append(ask_question(q))


Question: What is the official name of India?
Answer: Republic of India

Question: Where is India located geographically?
Answer: to the north of the equator between 6° 44′ and 35° 30′ north latitude[m] and 68° 7′ and 97° 25′ east longitude

Question: How large is India compared to other countries?
Answer: seventh-largest country by area

Question: When did India gain independence?
Answer: 1947

Question: What seas and oceans surround India?
Answer: Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast

Question: Which countries share land borders with India?
Answer: Pakistan to the west;[k] China, Nepal, and Bhutan to the north; and Bangladesh and Myanmar to the east

Question: What is the population of India as of 2023?
Answer: 1,428,627,663

Question: What are the major languages spoken in India?
Answer: Indo-Aryan languages

Question: How is India politically structured?
Answer: India is a parliamentary republic with a multi-party syst

In [ ]:
print(candidates)

['Republic of India', 'to the north of the equator between 6° 44′ and 35° 30′ north latitude[m] and 68° 7′ and 97° 25′ east longitude', 'seventh-largest country by area', '1947', 'Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast', 'Pakistan to the west;[k] China, Nepal, and Bhutan to the north; and Bangladesh and Myanmar to the east', '1,428,627,663', 'Indo-Aryan languages', 'India is a parliamentary republic with a multi-party system', 'parliamentary system', 'Narendra Modi', 'caste had emerged within Hinduism,[35] and Buddhism and Jainism had arisen, proclaiming social orders unlinked to heredity', 'Hinduism (79.80% of the population), followed by Islam', 'the first urban culture in South Asia', 'Mughal Empire', 'The rights promised to Indians were granted slowly,[51][52] but technological changes were introduced, and modern ideas of introduced not long after their introduction in Europe', '28 states and 8 union territories', '74.04%

In [ ]:
evaluate_rouge(candidates, references)

ROUGE evaluation results:
ROUGE-1 (F1): 0.4950
ROUGE-2 (F1): 0.3027
ROUGE-L (F1): 0.4829


baseline generation

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

def generate_answer(question, max_new_tokens=150):
    inputs = tokenizer(question, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        num_beams=4,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

baseline_answers = []
for q in question:
    ans = ask_question(q)
    baseline_answers.append(ans)



Question: What is the official name of India?
Answer: the name "India" is derived from the Classical Latin India, a reference to South Asia and an uncertain region to its east. In turn "India" derived successively from Hellenistic Greek India (), Ancient Greek Indos (), Old Persian Hindush (an eastern province of the Achaemenid Empire), and ultimately its cognate, the Sanskrit Sindhu, or 'river'—specifically the Indus River, and by extension its well-settled southern basin.

Question: Where is India located geographically?
Answer: to the north of the equator between 6° 44′ and 35° 30′ north latitude[m] and 68° 7′ and 97° 25′ east longitude

Question: How large is India compared to other countries?
Answer: seventh-largest country by area

Question: When did India gain independence?
Answer: 1947

Question: What seas and oceans surround India?
Answer: Indian Ocean

Question: Which countries share land borders with India?
Answer: India is near Sri Lanka and the Maldives; its Andaman and N

In [ ]:
evaluate_rouge(baseline_answers, references)

ROUGE evaluation results:
ROUGE-1 (F1): 0.3969
ROUGE-2 (F1): 0.2385
ROUGE-L (F1): 0.3952


cleaning corpus

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def fetch_html(url):
    """Fetch HTML content from a URL."""
    response = requests.get(url)
    response.raise_for_status()  # Raise error if request fails
    return response.text

def clean_html(raw_html):
    """Remove HTML tags using BeautifulSoup."""
    soup = BeautifulSoup(raw_html, "html.parser")
    return soup.get_text(separator="\n")

def remove_references(text):
    """Remove citations and references like [23], [a], etc."""
    return re.sub(r"\[[^\]]*\]", "", text)

def normalize_whitespace(text):
    """Replace multiple spaces and newlines with single ones."""
    text = re.sub(r"\s+", " ", text)  # Replace multiple whitespace with single space
    text = re.sub(r" ?\n ?", "\n", text)  # Clean newlines surrounded by spaces
    return text.strip()

def preprocess_url(url):
    """Fetch and preprocess text from a URL."""
    raw_html = fetch_html(url)
    text = clean_html(raw_html)
    text = remove_references(text)
    text = normalize_whitespace(text)
    return text

# Example usage:
# if __name__ == "__main__":
url = "https://en.wikipedia.org/wiki/India"
cleaned_text = preprocess_url(url)

# Optional: split into paragraphs by double newlines (or customize)
paragraphs = [p.strip() for p in cleaned_text.split("\n\n") if len(p.strip()) > 100]

print(f"Total paragraphs extracted: {len(paragraphs)}")
for i, para in enumerate(paragraphs[:3]):
    print(f"\nParagraph {i+1}:\n{para}\n")

Total paragraphs extracted: 1

Paragraph 1:
India - Wikipedia Jump to content Main menu Main menu move to sidebar hide Navigation Main page Contents Current events Random article About Wikipedia Contact us Contribute Help Learn to edit Community portal Recent changes Upload file Special pages Search Search Appearance Donate Create account Log in Personal tools Donate Create account Log in Pages for logged out editors learn more Contributions Talk Contents move to sidebar hide (Top) 1 Etymology 2 History Toggle History subsection 2.1 Ancient India 2.2 Medieval India 2.3 Early modern India 2.4 Modern India 3 Geography Toggle Geography subsection 3.1 Biodiversity 4 Politics and government Toggle Politics and government subsection 4.1 Politics 4.2 Government 4.3 Administrative divisions 4.3.1 States 4.3.2 Union territories 5 Foreign, economic, and strategic relations 6 Economy Toggle Economy subsection 6.1 Industries 6.2 Energy 6.3 Socio-economic challenges 7 Demographics, languages, and r

In [ ]:
# Build RetrievalQA pipeline
qa_pipeline2 = RetrievalQA.from_chain_type(
    llm=llm,
    retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k":5, "fetch_k":20}),
    return_source_documents=True  # optional, for traceability
)

In [ ]:
def ask_question2(question: str):
    # Use 'invoke' to get the result when there are multiple output keys
    result = qa_pipeline2.invoke(question)

    print("\nQuestion:", question)
    print("Answer:", result['result'] if isinstance(result, dict) and 'result' in result else result)
    return result['result'] if isinstance(result, dict) and 'result' in result else result

In [ ]:
candidates2 = []
for q in question:
        candidates2.append(ask_question2(q))